In [1]:
from baselines import MLPClassifier, LinearClassifier
from easyeditor import KEIDDataset
from torch.utils.data import DataLoader, TensorDataset
from sklearn.linear_model import LogisticRegression
import argparse
import pickle
import logging
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2,3"
import torch
torch.manual_seed(42)
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['font.size'] = 12
from tqdm import tqdm
import numpy as np
import json

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
type_mapping = {
    "Non edited": 0,
    "Fact updating": 1,
    "Misinformation injection": 2,
    "Offensiveness injection": 3,
    "Behavioral misleading injection": 4,
    "Bias injection": 5
}
LOG = logging.getLogger(__name__)

/home/hello/miniconda3/envs/editdirection/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
args = argparse.Namespace(
    method='LogR',
    feature_dir='./features',
    log_level='INFO',
    rephrased=False,
)
test_set = KEIDDataset('./data/edit_intention_split/test.json', type_mapping)
train_set = KEIDDataset('./data/edit_intention_split/train.json', type_mapping)
edited_llms = ['llama3.1-8b','llama2-13b']
all_precisions = {}
all_recalls = {}
all_f1s = {}
for edited_llm in tqdm(edited_llms, total=len(edited_llms)):
    all_precisions[edited_llm] = {}
    all_recalls[edited_llm] = {}
    all_f1s[edited_llm] = {}
    layer_num = 32 if edited_llm=='llama3.1-8b' else 40
    for edit_method in ['ft','grace','unke']:
        all_precisions[edited_llm][edit_method] = []
        all_recalls[edited_llm][edit_method] = []
        all_f1s[edited_llm][edit_method] = []
        
        LOG.info("Preprocess features..")
        test_feature_dir = f"{args.feature_dir}/all_hs/{edit_method}_{edited_llm}_testset{'_rephrased' if args.rephrased else ''}.pkl"
        if os.path.isfile(test_feature_dir):
            with open(test_feature_dir, 'rb') as f:
                test_feature = pickle.load(f)
        else:
            raise ValueError(f"File {test_feature_dir} isn't exsit.")
        train_feature_dir = f"{args.feature_dir}/all_hs/{edit_method}_{edited_llm}_trainset{'_rephrased' if args.rephrased else ''}.pkl"
        if os.path.isfile(train_feature_dir):
            with open(train_feature_dir, 'rb') as f:
                train_feature = pickle.load(f)
        else:
            raise ValueError(f"File {train_feature_dir} isn't exsit.")

        
        all_test_hs = test_feature['all_hidden_states']
        all_train_hs = train_feature['all_hidden_states']
        testset_labels = np.array([label for _,_, label in test_set])
        for layer in range(0,layer_num):
            np_train_hs = []
            for hs in all_train_hs:
                np_train_hs.append(hs[layer][0,-1,:].numpy())
            np_train_hs = np.array(np_train_hs)

            np_test_hs = []
            for hs in all_test_hs:
                np_test_hs.append(hs[layer][0,-1,:].numpy())
            np_test_hs = np.array(np_test_hs)

            
            y = np.array([label for _,_, label in train_set])

            if args.method == "LDA":
                model = LDA()
            elif args.method == "LogR":
                model = LogisticRegression( solver='lbfgs', max_iter=1000)
            else:
                raise NotImplementedError(f"{args.method} not implemented")
            LOG.info(f"Use {args.method} classifier")
            model.fit(np_train_hs, y)
            predictions = model.predict(np_test_hs)
            precision = precision_score(testset_labels, predictions, average='macro')
            recall = recall_score(testset_labels, predictions, average='macro')
            f1 = f1_score(testset_labels, predictions, average='macro')
            all_precisions[edited_llm][edit_method].append(precision)
            all_recalls[edited_llm][edit_method].append(recall)
            all_f1s[edited_llm][edit_method].append(f1)
            eval_res = f"{edited_llm}-{edit_method}-{layer_num}\n" + f'Precision: {precision:.3f}\n' + f'Recall: {recall:.3f}\n' + f'F1 Score: {f1:.3f}'
            print(eval_res)

allres = {"Precision": all_precisions, "Recall": all_recalls, "F1": all_f1s}
with open("./results/ablation_open_res.json", 'w') as f:
    json.dump(allres, f, indent=4)

### Plot results of ablation study of openllm

In [ ]:

import json
import matplotlib.pyplot as plt
import seaborn as sns

# 定义绘图函数
def plot_metrics(data):
    # 设置样式
    sns.set(style='whitegrid')

    # 创建一个新的图表
    plt.figure(figsize=(18, 5))
    title_font = {'family': 'Times New Roman', 'size': 18, 'weight': 'bold'}  # 标题字体
    label_font = {'family': 'Times New Roman', 'size': 18}  # 坐标轴标签字体
    legend_font = {'family': 'Times New Roman', 'size': 18}  # 图例字体
    # 遍历每个指标（Precision, Recall, F1）
    for i, metric in enumerate(data.keys()):
        # 创建一个子图
        plt.subplot(1, 3, i+1)
        
        # 遍历每个模型
        for model in data[metric].keys():
            # 遍历每个方法（ft, grace, unke）
            for method in data[metric][model].keys():
                # 获取 y 值
                y = data[metric][model][method]
                # 获取 x 值（层序号）
                x = list(range(len(y)))
                llm = "Llama3.1-8B" if model=="llama3.1-8b" else "Llama2-13B"
                if method == "unke":
                    meth = "UnKE"
                elif method == 'ft':
                    meth = "FT-M"
                else:
                    meth = method.upper()
                # 绘制折线图
                plt.plot(x, y, marker='o' if model=="llama3.1-8b" else 'D', label=f'{llm}-{meth}', linestyle='-' if model=="llama3.1-8b" else '--')

        # 设置标题和标签
        plt.title(f'{metric} of LogR across LLMs and editing methods',fontdict=title_font)
        plt.xlabel('Layer Index', fontdict=label_font)
        plt.ylabel(metric, fontdict=label_font)
        plt.legend(loc='best', prop=legend_font)

    # 调整布局
    plt.tight_layout()
    plt.savefig("./figs/ablation_LogR.pdf")

# 调用函数进行绘图

allres = json.load(open("./results/ablation_open_res.json"))
plot_metrics(allres)
